In [1]:
import pandas as pd

In [2]:
# Charger le fichier Excel
df_macro_raw= pd.read_excel("data\macro_data\Données_macro_hist_v2.xlsx")

# Preprocessing du dataframe
df = df_macro_raw.copy()

id_cols = ["Region", "Variable", "Unit"]
value_cols = [c for c in df.columns if c not in id_cols]

df_long = df.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="quarter",
    value_name="value"
)


Nous allons récupérer les variables macroéconomiques à considérer pour la modélisation des facteurs systémiques. D'abord, au regard des données dont nous disposons, nous ne pouvons considérer que les variables provenant des régions d'EUrope (5 variables disponibles) et des US (9 variables disponibles).

In [3]:
# Sélection des régions
regions_keep = ["United States"]
df = df_macro_raw[df_macro_raw["Region"].isin(regions_keep)].copy()
# Colonnes temporelles
time_cols = [c for c in df.columns if "-" in c]
# On garde uniquement les lignes sans NaN sur toute la période
df = df.dropna(subset=time_cols)
# Mapping régions → suffixes
region_map = {
    "United States": "US"
}

def clean_var_name(var):
    return (
        var.strip()
           .replace(" ", "_")
           .replace("(", "")
           .replace(")", "")
           .replace("/", "_")
    )

df["var_name"] = (
    df["Region"].map(region_map)
    + "_"
    + df["Variable"].apply(clean_var_name)
)


# Passage au format long
df_long = df.melt(
    id_vars=["var_name"],
    value_vars=time_cols,
    var_name="date",
    value_name="value"
)

# Pivot final
df_ts = df_long.pivot(
    index="date",
    columns="var_name",
    values="value"
).sort_index()

# Conversion "2010-Q1" → PeriodIndex trimestriel
df_ts.index = pd.PeriodIndex(df_ts.index, freq="Q").to_timestamp()

## STATIONARITÉ

### TESTS DE STATIONNARITE

In [4]:
import numpy as np
import pandas as pd
import warnings

from statsmodels.tsa.stattools import adfuller, kpss

def stationarity_tests_summary(
    df: pd.DataFrame,
    variables=None,
    adf_alpha: float = 0.05,
    kpss_alpha: float = 0.05,
    kpss_regression: str = "c",
    autolag: str = "AIC",
    dropna: bool = True,
    min_n: int = 20
) -> pd.DataFrame:
    """
    Calcule ADF + KPSS pour une liste de variables, renvoie un tableau récapitulatif
    avec p-values, décisions à 5% et diagnostic.

    Parameters
    ----------
    df : DataFrame (wide) avec colonnes = variables.
    variables : list[str] ou None (par défaut: toutes les colonnes numériques).
    adf_alpha : seuil de rejet ADF (H0 = non-stationnaire).
    kpss_alpha : seuil de rejet KPSS (H0 = stationnaire).
    kpss_regression : 'c' (stationnaire autour d'une constante) ou 'ct' (constante+tendance).
    autolag : méthode de sélection du lag pour ADF ('AIC','BIC','t-stat',None).
    dropna : supprime les NA avant tests.
    min_n : taille minimale requise pour lancer les tests.

    Returns
    -------
    DataFrame avec colonnes:
    variable, n, adf_pvalue, kpss_pvalue, ADF_stationary_5pct, KPSS_stationary_5pct, diagnosis
    """
    if variables is None:
        # par défaut: toutes les colonnes numériques
        variables = df.select_dtypes(include=[np.number]).columns.tolist()
    else:
        variables = list(variables)

    results = []

    for var in variables:
        if var not in df.columns:
            results.append({
                "variable": var, "n": 0,
                "adf_pvalue": np.nan, "kpss_pvalue": np.nan,
                "ADF_stationary_5pct": np.nan, "KPSS_stationary_5pct": np.nan,
                "diagnosis": "Missing"
            })
            continue

        s = df[var]
        s = s.dropna() if dropna else s

        # Cast float (évite certains soucis)
        s = pd.to_numeric(s, errors="coerce").dropna()

        n = len(s)

        # Cas série trop courte / constante
        if n < min_n or s.nunique() <= 1:
            results.append({
                "variable": var, "n": n,
                "adf_pvalue": np.nan, "kpss_pvalue": np.nan,
                "ADF_stationary_5pct": np.nan, "KPSS_stationary_5pct": np.nan,
                "diagnosis": "Insufficient/Constant"
            })
            continue

        # -------- ADF --------
        adf_p = np.nan
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                adf_out = adfuller(s.values, autolag=autolag)
            adf_p = float(adf_out[1])
        except Exception:
            adf_p = np.nan

        # -------- KPSS --------
        kpss_p = np.nan
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                kpss_out = kpss(s.values, regression=kpss_regression, nlags="auto")
            kpss_p = float(kpss_out[1])
        except Exception:
            kpss_p = np.nan

        # Décisions
        # ADF: rejeter H0 (non-stationnaire) si p < alpha => stationnaire
        adf_stationary = (adf_p < adf_alpha) if not np.isnan(adf_p) else np.nan
        # KPSS: rejeter H0 (stationnaire) si p < alpha => non stationnaire
        # donc stationnaire si p > alpha
        kpss_stationary = (kpss_p > kpss_alpha) if not np.isnan(kpss_p) else np.nan

        # Diagnostic combiné
        if adf_stationary is True and kpss_stationary is True:
            diag = "Stationary"
        elif adf_stationary is False and kpss_stationary is False:
            diag = "Non-stationary"
        elif (adf_stationary is np.nan) or (kpss_stationary is np.nan):
            diag = "Test failed"
        else:
            diag = "Ambiguous"

        results.append({
            "variable": var,
            "n": n,
            "adf_pvalue": adf_p,
            "kpss_pvalue": kpss_p,
            "ADF_stationary_5pct": adf_stationary,
            "KPSS_stationary_5pct": kpss_stationary,
            "diagnosis": diag
        })

    out = pd.DataFrame(results)

    # Optionnel: trier pour lecture (Non-stationary / Ambiguous / Stationary)
    order = pd.CategoricalDtype(
        categories=["Non-stationary", "Ambiguous", "Stationary", "Insufficient/Constant", "Missing", "Test failed"],
        ordered=True
    )
    if "diagnosis" in out.columns:
        out["diagnosis"] = out["diagnosis"].astype(order)

    return out


In [5]:
variables = df_ts.columns.tolist()
stationarity_df = stationarity_tests_summary(df_ts, variables)
stationarity_df


,variable,n,adf_pvalue,kpss_pvalue,ADF_stationary_5pct,KPSS_stationary_5pct,diagnosis
0,US_Central_bank_Intervention_rate_policy_inter...,61,9.969745e-01,0.010000,False,False,Non-stationary
1,US_Effective_exchange_rate,61,9.442045e-01,0.010000,False,False,Non-stationary
2,US_Equity_prices,61,9.988414e-01,0.010000,False,False,Non-stationary
3,US_GDP_Growth_Rate,61,7.219137e-11,0.100000,True,True,Stationary
4,US_House_prices_residential,61,9.306144e-01,0.010000,False,False,Non-stationary
5,US_Inflation_rate,61,5.391757e-01,0.049518,False,False,Non-stationary
6,US_Long_term_interest_rate,61,5.324999e-01,0.100000,False,True,Ambiguous
7,US_Oil_price,61,2.953302e-01,0.100000,False,True,Ambiguous
8,US_Unemployment_rate,61,6.014751e-02,0.010000,False,False,Non-stationary


In [6]:
from statsmodels.tsa.filters.hp_filter import hpfilter

def hp_gap(series, lamb=1600):
    """
    HP gap (cycle) pour données trimestrielles.
    Retourne une série alignée sur l'index original.
    """
    s = series.astype(float)
    cycle, trend = hpfilter(s.dropna(), lamb=lamb)
    return cycle.reindex(series.index)

In [7]:
df_ts["US_Unemployment_rate_hp_gap"] = hp_gap(df_ts["US_Unemployment_rate"], lamb=1600)
df_ts["US_Unemployment_rate_hp_gap_diff"] = df_ts["US_Unemployment_rate_hp_gap"].diff()
df_ts["US_Long_term_interest_rate_gap"] = hp_gap(df_ts["US_Long_term_interest_rate"], lamb=1600)
df_ts["US_Long_term_interest_rate_gap_diff"] = df_ts["US_Long_term_interest_rate_gap"].diff()
df_ts["US_House_prices_residential_hp_gap"] = hp_gap(df_ts["US_House_prices_residential"], lamb=1600)
df_ts["US_House_prices_residential_hp_gap_diff"] = df_ts["US_House_prices_residential_hp_gap"].diff()
df_ts["US_Effective_exchange_rate_hp_gap"] = hp_gap(df_ts["US_Effective_exchange_rate"], lamb=1600)
df_ts["US_Effective_exchange_rate_hp_gap_diff"] = df_ts["US_Effective_exchange_rate_hp_gap"].diff()
df_ts["US_Central_bank_Intervention_rate_policy_interest_rate_diff"] = df_ts["US_Central_bank_Intervention_rate_policy_interest_rate"].diff()



import numpy as np

def safe_log(series):
    s = series.astype(float)
    return np.log(s.where(s > 0))
df_ts["US_Equity_prices_log"] = safe_log(df_ts["US_Equity_prices"])
df_ts["US_Equity_prices_log_hp_gap"] = hp_gap(df_ts["US_Equity_prices_log"], lamb=1600)

df_ts["US_Oil_price_log"] = safe_log(df_ts["US_Oil_price"])
df_ts["US_Oil_price_log_hp_gap"] = hp_gap(df_ts["US_Oil_price_log"], lamb=1600)
df_ts["US_GDP_log"]= safe_log(df_ts["US_GDP_Growth_Rate"])
df_ts["US_GDP_log_hp_gap"] = hp_gap(df_ts["US_GDP_log"], lamb=1600)
df_ts["US_GDP_log_hp_gap_diff"] = df_ts["US_GDP_log_hp_gap"].diff()

variables = df_ts.columns.tolist()
stationarity_df = stationarity_tests_summary(df_ts, variables)
stationarity_df


stationary_cols = stationarity_df.loc[
    (stationarity_df["diagnosis"]) != "Non-stationary",
    "variable"
].tolist()

df_ts1=df_ts[stationary_cols]


c:\Users\werid\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  trend = spsolve(I+lamb*K.T.dot(K), x, use_umfpack=use_umfpack)
c:\Users\werid\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  trend = spsolve(I+lamb*K.T.dot(K), x, use_umfpack=use_umfpack)
c:\Users\werid\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  trend = spsolve(I+lamb*K.T.dot(K), x, use_umfpack=use_umfpack)
c:\Users\werid\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  trend = spsolve(I+lamb*K.T.dot(K), x, use_umfpack=use_u

In [8]:
df_macro_stationary = df_ts1.copy()
df_macro_stationary

var_name,US_GDP_Growth_Rate,US_Long_term_interest_rate,US_Oil_price,US_Unemployment_rate_hp_gap,US_Unemployment_rate_hp_gap_diff,US_Long_term_interest_rate_gap,US_Long_term_interest_rate_gap_diff,US_House_prices_residential_hp_gap,US_House_prices_residential_hp_gap_diff,US_Effective_exchange_rate_hp_gap,US_Effective_exchange_rate_hp_gap_diff,US_Central_bank_Intervention_rate_policy_interest_rate_diff,US_Equity_prices_log_hp_gap,US_Oil_price_log,US_Oil_price_log_hp_gap,US_GDP_log,US_GDP_log_hp_gap,US_GDP_log_hp_gap_diff
date,,,,,,,,,,,,,,,,,,
2010-01-01,0.484501,3.716667,76.674837,-0.144786,NaN,0.604406,NaN,6.085035,NaN,3.738163,NaN,NaN,0.066740,4.339574,-0.261943,-0.724636,-0.239980,NaN
2010-04-01,0.967586,3.490000,78.845036,-0.130507,0.014279,0.457244,-0.147162,5.412524,-0.672511,4.980603,1.242440,0.000000,-0.089092,4.367484,-0.237491,-0.032951,0.466723,0.706703
2010-07-01,0.771085,2.786667,76.675000,-0.082804,0.047703,-0.166962,-0.624206,3.601028,-1.811496,3.670707,-1.309896,0.000000,-0.016866,4.339576,-0.268695,-0.259957,0.254885,-0.211838
2010-10-01,0.525110,2.863333,87.033160,0.165162,0.247966,-0.012209,0.154753,2.021698,-1.579330,0.003025,-3.667682,0.000000,0.050658,4.466289,-0.144801,-0.644147,-0.114129,-0.369014
2011-01-01,-0.237205,3.460000,105.369424,-0.086385,-0.251547,0.660943,0.673152,-0.815040,-2.836737,-1.193519,-1.196544,0.000000,0.073823,4.657472,0.044519,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01,0.404802,4.160000,81.955714,-0.183095,0.109214,0.328628,-0.450068,-0.542648,-0.920350,-1.111397,-1.290035,0.000000,0.040896,4.406179,-0.009041,-0.904357,-0.544293,-0.648202
2024-04-01,0.738980,4.443333,84.981986,0.057483,0.240578,0.442144,0.113516,-1.444021,-0.901372,0.289890,1.401287,0.000000,0.049622,4.442439,0.015416,-0.302485,0.076284,0.620577
2024-07-01,0.759510,3.946667,78.708317,0.296755,0.239272,-0.224024,-0.666168,-2.201167,-0.757147,-1.039576,-1.329466,-0.166667,0.073607,4.365749,-0.072583,-0.275081,0.122421,0.046136


In [13]:
df_ts1.to_csv("data/macro_data/df_macro_stationary.csv", index=True)

In [12]:
stationarity_df


,variable,n,adf_pvalue,kpss_pvalue,ADF_stationary_5pct,KPSS_stationary_5pct,diagnosis
0,US_Central_bank_Intervention_rate_policy_inter...,61,9.969745e-01,0.010000,False,False,Non-stationary
1,US_Effective_exchange_rate,61,9.442045e-01,0.010000,False,False,Non-stationary
2,US_Equity_prices,61,9.988414e-01,0.010000,False,False,Non-stationary
3,US_GDP_Growth_Rate,61,7.219137e-11,0.100000,True,True,Stationary
4,US_House_prices_residential,61,9.306144e-01,0.010000,False,False,Non-stationary
5,US_Inflation_rate,61,5.391757e-01,0.049518,False,False,Non-stationary
6,US_Long_term_interest_rate,61,5.324999e-01,0.100000,False,True,Ambiguous
7,US_Oil_price,61,2.953302e-01,0.100000,False,True,Ambiguous
8,US_Unemployment_rate,61,6.014751e-02,0.010000,False,False,Non-stationary
9,US_Unemployment_rate_hp_gap,61,1.375126e-04,0.100000,True,True,Stationary
